In [ ]:
# calculate inception score for cifar-10 in Keras
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy.random import shuffle
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize
from numpy import asarray

def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		new_image = resize(image, new_shape, 0)
		images_list.append(new_image)
	return asarray(images_list)
 

def calculate_inception_score(images, n_split=10, eps=1E-16):
	model = InceptionV3()
	scores = list()
	n_part = floor(images.shape[0] / n_split)
	for i in range(n_split):
		ix_start, ix_end = i * n_part, (i+1) * n_part
		subset = images[ix_start:ix_end]
		subset = subset.astype('float32')
		subset = scale_images(subset, (299,299,3))
		subset = preprocess_input(subset)
		p_yx = model.predict(subset)
		p_y = expand_dims(p_yx.mean(axis=0), 0)
		kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
		sum_kl_d = kl_d.sum(axis=1)
		avg_kl_d = mean(sum_kl_d)
		is_score = exp(avg_kl_d)
		scores.append(is_score)
	is_avg, is_std = mean(scores), std(scores)
	return is_avg, is_std
 


loaded (1000, 32, 32, 3)


In [ ]:
import os
import cv2
path_img = './Image_Gennerator_byDCGAN'
list_file = os.listdir(path_img)


In [ ]:
from tqdm import tqdm
images = []
for i in tqdm(range(1000)):
  filename = os.path.join(path_img, list_file[i])
  img = cv2.imread(filename)
  images.append(img)

100%|██████████| 1000/1000 [00:13<00:00, 74.04it/s]


In [ ]:
import numpy as np
images = np.array(images)

In [ ]:
shuffle(images)
is_avg, is_std = calculate_inception_score(images)
print('score', is_avg, is_std)

score 2.0260053 0.08950541
